In [43]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10 # Adjust this number based on your data
n_nonzero_coefs = 5  # Number of non-zero coefficients in sparse coding

In [44]:
def create_sparse_representation(data, n_components, n_nonzero_coefs):
    """
    Create a sparse representation with reduced dimensions.
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Reduced dimensionality
    - n_nonzero_coefs: Sparsity level
    
    Returns:
    - Sparse representation with reduced dimensions.
    """
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)

        # Learn a dictionary (or use a pre-defined one)
        dict_learner = DictionaryLearning(
            n_components=n_components, 
            transform_algorithm='lasso_lars', 
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        dict_learner.fit(feature_data)
        sparse_codes = dict_learner.transform(feature_data)

        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes

    return sparse_representation

In [45]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_alice_real = train_alice_cirs[..., 0]  # Shape: (100, 251)
train_alice_imaginary = train_alice_cirs[..., 1]  # Shape: (100, 251)
train_alice_magnitude = np.sqrt(np.maximum(train_alice_real**2 + train_alice_imaginary**2, 0))
train_alice_magnitude = train_alice_magnitude[..., np.newaxis]
# train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])

train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_eve_real = train_alice_cirs[..., 0]  # Shape: (100, 251)
train_eve_imaginary = train_alice_cirs[..., 1]  # Shape: (100, 251)
train_eve_magnitude = np.sqrt(np.maximum(train_eve_real**2 + train_eve_imaginary**2, 0))
train_eve_magnitude = train_eve_magnitude[..., np.newaxis]

train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))
train_sparse_cirs = create_sparse_representation(train_data_combined, n_components, n_nonzero_coefs)
train_sparse_cirs = train_sparse_cirs.reshape(train_sparse_cirs.shape[0], -1)
print(train_sparse_cirs.shape)
# # Learn the dictionary
# dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs)
# dict_learner.fit(train_data_combined)
# # D = dict_learner.components_  # Shape: (n_components, feature_dim)


# train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
# train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
# train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
# print(train_alice_sparse_codes.shape)

(1600, 10)


In [46]:
# ----------------- Test data -----------------
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_alice_real = test_alice_cirs[..., 0]  # Shape: (100, 251)
test_alice_imaginary = test_alice_cirs[..., 1]  # Shape: (100, 251)
test_alice_magnitude = np.sqrt(np.maximum(test_alice_real**2 + test_alice_imaginary**2, 0))
test_alice_magnitude = test_alice_magnitude[..., np.newaxis]

test_eve_cirs = testCIR[:, eve_channel, :, :]
test_eve_real = test_eve_cirs[..., 0]  # Shape: (100, 251)
test_eve_imaginary = test_eve_cirs[..., 1]  # Shape: (100, 251)
test_eve_magnitude = np.sqrt(np.maximum(test_eve_real**2 + test_eve_imaginary**2, 0))
test_eve_magnitude = test_eve_magnitude[..., np.newaxis]

# test amplitude
# test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
# test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs_combined = np.vstack((test_alice_magnitude, test_eve_magnitude))
test_sparse_cirs = create_sparse_representation(test_cirs_combined, n_components, n_nonzero_coefs)
test_sparse_cirs = test_sparse_cirs.reshape(test_sparse_cirs.shape[0], -1)
print(test_sparse_cirs.shape)
# Transform test data into sparse codes
# test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
# test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# # Combine the sparse codes
# test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
# print('Test data shape:', test_sparse_codes.shape)

(400, 10)


In [47]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))
print(train_labels.shape)

# Labels for test data
test_alice_labels = np.zeros(test_alice_cirs.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_cirs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))
print(test_labels.shape)


(1600,)
(400,)


In [48]:
# # Dictionary
# D = train_sparse_codes.T
# print('Dictionary shape:', D.shape)

In [49]:
scaler = StandardScaler()
train_sparse_cirs = scaler.fit_transform(train_sparse_cirs)
print(train_sparse_cirs.shape)
# Apply the same scaling to test data
test_sparse_cirs = scaler.transform(test_sparse_cirs)
print(test_sparse_cirs.shape)

(1600, 10)
(400, 10)


In [50]:
# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_sparse_cirs, train_labels)

# Predict on test data
predictions = classifier.predict(test_sparse_cirs)

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 49.25%
tp: 98
tn: 99
fp: 101
fn: 102
MDR: 0.505
FAR: 0.51
AR: 0.4925
